<a href="https://colab.research.google.com/drive/1aGZv3bNDvrEOOCq78lSYREGiO7jpTgt7?usp=sharing" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

#### [Huggingface Models](https://huggingface.co/) - Hugging Face models – Requires T4 GPU. Select ‘Change Runtime’ in Colab to enable.

### Install required libraries

In [ ]:
!pip install --upgrade --quiet  langchain-huggingface text-generation transformers langchainhub bitsandbytes accelerate

### Import related libraries

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
import getpass

### 🔑 Provide Huggingface API Key

[Huggingface API Generation Key Link](https://huggingface.co/docs/hub/security-tokens)


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()

··········


In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        return_full_text=False,
    ),
    model_kwargs={"quantization_config": quantization_config},
)

chat_model = ChatHuggingFace(llm=llm)

In [ ]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

ai_msg = chat_model.invoke(messages)
print(ai_msg.content)